In [1]:
# read it in to inspect it
with open('../data/tinyshakespeare.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [2]:
# from datasets import load_dataset

# ds = load_dataset("Blackroot/Tiny-Open-Domain-Books")

# text = str(ds.data['train']['text'])[11:]

# # write 
# with open('../data/tinystories.txt', 'w', encoding='utf-8') as f:
#     f.write(text)

In [3]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [4]:
print(text[:500])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


In [5]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [6]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [7]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.dtype, data.shape)
data[:10]

torch.int64 torch.Size([1115394])


tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47])

In [8]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [9]:
block_size = 8 # context length

In [10]:
x = train_data[:block_size]
y = train_data[:block_size+1]
for i in range(block_size):
    context = x[:i+1]
    print(context, y[i+1])

tensor([18]) tensor(47)
tensor([18, 47]) tensor(56)
tensor([18, 47, 56]) tensor(57)
tensor([18, 47, 56, 57]) tensor(58)
tensor([18, 47, 56, 57, 58]) tensor(1)
tensor([18, 47, 56, 57, 58,  1]) tensor(15)
tensor([18, 47, 56, 57, 58,  1, 15]) tensor(47)
tensor([18, 47, 56, 57, 58,  1, 15, 47]) tensor(58)


In [11]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8 

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size , (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

In [12]:
Xb, Yb = get_batch('train')
print('input', Xb.shape, Xb)
print('output', Yb.shape, Yb)
for b in range(batch_size): 
    for t in range(block_size): # time dimension
        context = Xb[b, :t+1]
        target = Yb[b, t]
        print(f'Context: {context} Target: {target}')

input torch.Size([4, 8]) tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
output torch.Size([4, 8]) tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
Context: tensor([24]) Target: 43
Context: tensor([24, 43]) Target: 58
Context: tensor([24, 43, 58]) Target: 5
Context: tensor([24, 43, 58,  5]) Target: 57
Context: tensor([24, 43, 58,  5, 57]) Target: 1
Context: tensor([24, 43, 58,  5, 57,  1]) Target: 46
Context: tensor([24, 43, 58,  5, 57,  1, 46]) Target: 43
Context: tensor([24, 43, 58,  5, 57,  1, 46, 43]) Target: 39
Context: tensor([44]) Target: 53
Context: tensor([44, 53]) Target: 56
Context: tensor([44, 53, 56]) Target: 1
Context: tensor([44, 53, 56,  1]) Target: 58
Context: tensor([44, 53, 56,  1, 58]) Target: 46
Context: tensor([44, 53, 56,  1, 

In [13]:
# Bigram Language Model
import torch
import torch.nn as nn
from torch.nn  import functional as F
torch.manual_seed(1337)


class BigramLangaugeModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        # create embedding
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)   # (B,T,C)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            # focus only on the las 
            logits = logits[:, -1, :] # becomes (B,C)
            # apply the softmax to get probablities
            probs = F.softmax(logits, dim=1)     # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append the sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLangaugeModel(vocab_size)
logits, loss = model(Xb, Yb)
print(Xb.shape, Yb.shape)
print(logits.shape)
print(loss)
print(decode(model.generate(idx=torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([4, 8]) torch.Size([4, 8])
torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

SKIcLT;AcELMoTbvZv C?nq-QE33:CJqkOKH-q;:la!oiywkHjgChzbQ?u!3bLIgwevmyFJGUGp
wnYWmnxKWWev-tDqXErVKLgJ


In [14]:
# create a pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [15]:
max_iteration = 100000
batch_size = 32
for steps in range(100000):
    
    # sample batch from the data
    xb, yb = get_batch('train')
    
    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

2.5319576263427734


In [16]:
print(decode(model.generate(idx=torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))



Ofows ht IUS:
S:

ING flvenje ssutefr,
M:
War cl igagimous pray whars:
Panalit I It aithit terised 


## The mathematical trick in self attention
##### (tokens need to start talking to each other but only the previous one)

In [17]:
torch.manual_seed(1337)
B, T, C = 4, 8 , 2
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [18]:
# easiest way to interpret previous information is to average it
# we want x[b,t ] = mean_{i<=t} x[b,i]
xbow = torch.zeros(B, T, C)
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (T,C)
        xbow[b,t] = torch.mean(xprev, 0)


In [19]:
x[0]

tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])

In [20]:
xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [21]:
torch.manual_seed(42)
a = torch.ones(3,3)
b = torch.randint(0,10, (3,2)).float()
c = a @ b
print('a=')
print(a)
print('---')
print('b=')
print(b)
print('---')
print('c=')
print(c)

a=
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
---
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
---
c=
tensor([[14., 16.],
        [14., 16.],
        [14., 16.]])


In [22]:
torch.tril(torch.ones(3,3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [23]:
# Another way to calculate previous token mean is with diagonal matrix 

torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10, (3,2)).float()
c = a @ b
print('a=')
print(a)
print('---')
print('b=')
print(b)
print('---')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
---
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
---
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [24]:
# now we can use this trick to vectorize the for loop 
# version 2
wei = torch.tril(torch.ones(T,T))
wei = wei/wei.sum(1, keepdim=True)
xbow2 = wei @ x # (T,T) x (B,T,C) --> (B, T, T) x (B, T, C) --> (B, T, C)
torch.allclose(xbow, xbow2)

True

In [25]:
# version3
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))      # starts wiht interaction strenght of zero i.e. how much token from past/future we can to aggregate

wei = wei.masked_fill(tril == 0 , float('-inf'))   # this represents token from future can not interact
print(wei)
wei = F.softmax(wei, dim=1)   # e^x / sum(e^x)

xbow3 = wei @ x
torch.allclose(xbow, xbow3)

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])


True

In [26]:
# version 4 self attention
torch.manual_seed(1337)
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0 , float('-inf'))
wei = F.softmax(wei, dim=1)
out = wei @ x
out.shape

torch.Size([4, 8, 32])

In [27]:
# now we introduce self attension, so each token/node contain key and query vector,
# key represents what information token contains and query represents what information token is looking for. 
# so we can do dot product of query of current token with keys of all the previous token. this way alligned tokens/nodes will interact in higher amount 

# let's see a single head perform self attention

torch.manual_seed(1337)
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)  # (B, T, 16)
q = query(x) # (B, T, 16)
wei = q @ k.transpose(-2, -1) # (B, T, 16) x (B, 16, T) --> (B, T, T)

tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0 , float('-inf'))   # decoder version, if encoder this is not needed and all nodes will talk to each other
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
out.shape


# attention supports aribitary connectivity between nodes so it just does not have to be only past or future past which could be case of sentiment analysis

torch.Size([4, 8, 16])

In [28]:
# softmax can peak if values are not closer to zero i.e. if some values are very high or very low
# so need to device by square root of headsize to keep variance 1

k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)
wei = q @ k.transpose(-2, -1)

In [29]:
k.var(), q.var(), wei.var()

(tensor(1.0449), tensor(1.0700), tensor(17.4690))

In [30]:
k = torch.randn(B, T, head_size)
q = torch.randn(B, T, head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [31]:
k.var(), q.var(), wei.var()

(tensor(0.9006), tensor(1.0037), tensor(0.9957))